<a href="https://colab.research.google.com/github/syedanida/ieee-fraud-detection/blob/main/Fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autogluon
!pip install kaggle

In [2]:
from google.colab import files
files.upload()  # Use this to upload your kaggle.json file

Saving ieee-fraud-detection.zip to ieee-fraud-detection (1).zip


In [1]:
!ls -l /content

total 4
drwxr-xr-x 1 root root 4096 Sep 23 13:20 sample_data


In [2]:
!pwd

/content


In [3]:
!ls -l /content

total 4
drwxr-xr-x 1 root root 4096 Sep 23 13:20 sample_data


In [4]:
from google.colab import files
uploaded = files.upload()

Saving ieee-fraud-detection.zip to ieee-fraud-detection.zip


In [5]:
import zipfile
import os

# Path to the uploaded ZIP file
zip_path = '/content/ieee-fraud-detection.zip'
print(os.path.isfile(zip_path))
extract_dir = '/content/dataset/'

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Verify the extracted files
!ls -l /content/dataset


True
total 1323212
-rw-r--r-- 1 root root   6080314 Sep 24 20:55 sample_submission.csv
-rw-r--r-- 1 root root  25797161 Sep 24 20:55 test_identity.csv
-rw-r--r-- 1 root root 613194934 Sep 24 20:55 test_transaction.csv
-rw-r--r-- 1 root root  26529680 Sep 24 20:55 train_identity.csv
-rw-r--r-- 1 root root 683351067 Sep 24 20:55 train_transaction.csv


In [6]:
# List files in the directory
os.listdir(extract_dir)

['train_identity.csv',
 'train_transaction.csv',
 'test_transaction.csv',
 'test_identity.csv',
 'sample_submission.csv']

Loading the data into Colab

In [7]:
import pandas as pd

# Define file paths
extract_dir = '/content/dataset/'
train_identity_path = os.path.join(extract_dir, 'train_identity.csv')
train_transaction_path = os.path.join(extract_dir, 'train_transaction.csv')
test_identity_path = os.path.join(extract_dir, 'test_identity.csv')
test_transaction_path = os.path.join(extract_dir, 'test_transaction.csv')
sample_submission_path = os.path.join(extract_dir, 'sample_submission.csv')

# Load the datasets
train_identity = pd.read_csv(train_identity_path)
train_transaction = pd.read_csv(train_transaction_path)
test_identity = pd.read_csv(test_identity_path)
test_transaction = pd.read_csv(test_transaction_path)
sample_submission = pd.read_csv(sample_submission_path)

# Display the first few rows of each dataset
print("Train Identity Data:")
print(train_identity.head())
print("\nTrain Transaction Data:")
print(train_transaction.head())
print("\nTest Identity Data:")
print(test_identity.head())
print("\nTest Transaction Data:")
print(test_transaction.head())
print("\nSample Submission Data:")
print(sample_submission.head())


Train Identity Data:
   TransactionID  id_01     id_02  id_03  id_04  id_05  id_06  id_07  id_08  \
0        2987004    0.0   70787.0    NaN    NaN    NaN    NaN    NaN    NaN   
1        2987008   -5.0   98945.0    NaN    NaN    0.0   -5.0    NaN    NaN   
2        2987010   -5.0  191631.0    0.0    0.0    0.0    0.0    NaN    NaN   
3        2987011   -5.0  221832.0    NaN    NaN    0.0   -6.0    NaN    NaN   
4        2987016    0.0    7460.0    0.0    0.0    1.0    0.0    NaN    NaN   

   id_09  ...                id_31  id_32      id_33           id_34  id_35  \
0    NaN  ...  samsung browser 6.2   32.0  2220x1080  match_status:2      T   
1    NaN  ...   mobile safari 11.0   32.0   1334x750  match_status:1      T   
2    0.0  ...          chrome 62.0    NaN        NaN             NaN      F   
3    NaN  ...          chrome 62.0    NaN        NaN             NaN      F   
4    0.0  ...          chrome 62.0   24.0   1280x800  match_status:2      T   

  id_36 id_37  id_38  DeviceT

In [8]:
# Merge training datasets
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

# Merge test datasets
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

# Check merged data
print("Train Data:")
print(train_data.head())
print("\nTest Data:")
print(test_data.head())


Train Data:
   TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   

   card2  card3       card4  card5  ...                id_31  id_32  \
0    NaN  150.0    discover  142.0  ...                  NaN    NaN   
1  404.0  150.0  mastercard  102.0  ...                  NaN    NaN   
2  490.0  150.0        visa  166.0  ...                  NaN    NaN   
3  567.0  150.0  mastercard  117.0  ...                  NaN    NaN   
4  514.0  150.0  mastercard  102.0  ...  samsung browser 6.2   32.0   

       id_33           id_34  id_35 id_36 id_37  id_38  DeviceType  \
0        NaN             NaN

Setting up autogluon

In [9]:
import os

train_transaction_path = '/content/dataset/train_transaction.csv'
print(os.path.isfile(train_transaction_path))  # Should print True if file exists


True


In [ ]:
!pip install --upgrade autogluon pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
import pandas as pd
import os

# Define file paths
train_transaction_path = '/content/dataset/train_transaction.csv'

# Check if file exists
if not os.path.isfile(train_transaction_path):
    raise FileNotFoundError(f"File not found: {train_transaction_path}")

# Load the training data
train_data = pd.read_csv(train_transaction_path)

# Display the first few rows and columns of the dataset
print("First few rows of the dataset:")
print(train_data.head())
print("\nColumns in the dataset:")
print(train_data.columns)


First few rows of the dataset:
   TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   

   card2  card3       card4  card5  ... V330  V331  V332  V333  V334 V335  \
0    NaN  150.0    discover  142.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
1  404.0  150.0  mastercard  102.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
2  490.0  150.0        visa  166.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
3  567.0  150.0  mastercard  117.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
4  514.0  150.0  mastercard  102.0  ...  0.0   0.0   0.0   0.0   0.0  0.0   

  V336  V337  V338  V339  
0  NaN   NaN   N

In [2]:
print("Missing values per column:")
print(train_data.isnull().sum())

Missing values per column:
TransactionID          0
isFraud                0
TransactionDT          0
TransactionAmt         0
ProductCD              0
                   ...  
V335              508189
V336              508189
V337              508189
V338              508189
V339              508189
Length: 394, dtype: int64


In [1]:
!pip install autogluon

In [2]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# Define file paths
train_transaction_path = '/content/dataset/train_transaction.csv'

# Load a smaller subset of the training data
train_data = pd.read_csv(train_transaction_path, nrows=500)

# Define label and evaluation metric
label = 'isFraud'
eval_metric = 'roc_auc'
save_path = '/content/AutoGluonModels/'

# Train AutoGluon model
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='good_quality', time_limit=600)

# Check training results
results = predictor.fit_summary()
print(results)


Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          0
Memory Avail:       11.30 GB / 12.67 GB (89.1%)
Disk Space Avail:   60.41 GB / 107.72 GB (56.1%)
Presets specified: ['good_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True,
 'num_bag_sets': 1,
 'refit_full': True,
 'save_bag_folds': False,
 'set_best_to_refit_full': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.11111

(_ray_fit pid=6950) [50]	valid_set's binary_logloss: 0.0910878


(_ray_fit pid=7037) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=7037) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 2x across cluster]
(_ray_fit pid=7037)  [repeated 4x across cluster]
(_ray_fit pid=7037) You can install it with `pip install dask[dataframe]` or `conda install dask`. [repeated 2x across cluster]
(_ray_fit pid=7037) This will raise in a future version. [repeated 2x across cluster]
(_ray_fit pid=7037)   warnings.warn(msg, FutureWarning) [repeated 2x across cluster]


(_ray_fit pid=7037) [50]	valid_set's binary_logloss: 0.045526 [repeated 12x across cluster]


(_ray_fit pid=7117) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 2x across cluster]
(_ray_fit pid=7117) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 2x across cluster]
(_ray_fit pid=7117)  [repeated 4x across cluster]
(_ray_fit pid=7117) You can install it with `pip install dask[dataframe]` or `conda install dask`. [repeated 2x across cluster]
(_ray_fit pid=7117) This will raise in a future version. [repeated 2x across cluster]
(_ray_fit pid=7117)   warnings.warn(msg, FutureWarning) [repeated 2x across cluster]


...

(_ray_fit pid=8414) [50]	valid_set's binary_logloss: 0.0656376 [repeated 2x across cluster]


(_ray_fit pid=8502) /usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning:  [repeated 2x across cluster]
(_ray_fit pid=8502) Dask dataframe query planning is disabled because dask-expr is not installed. [repeated 2x across cluster]
(_ray_fit pid=8502) You can install it with `pip install dask[dataframe]` or `conda install dask`. [repeated 2x across cluster]
(_ray_fit pid=8502) This will raise in a future version. [repeated 2x across cluster]
(_ray_fit pid=8502)   warnings.warn(msg, FutureWarning) [repeated 2x across cluster]
(_ray_fit pid=8502)  [repeated 4x across cluster]


(_ray_fit pid=8502) [50]	valid_set's binary_logloss: 0.0228988 [repeated 12x across cluster]


(_dystack pid=6784) 	Warning: Exception caused LightGBM_BAG_L2 to fail during training... Skipping this model.
(_dystack pid=6784)   File "/usr/local/lib/python3.10/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 405, in _ray_fit
(_dystack pid=6784)     fold_model, pred_proba = _ray_predict_oof(
(_dystack pid=6784)   File "/usr/local/lib/python3.10/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 422, in _ray_predict_oof
(_dystack pid=6784)     fold_model.val_score = fold_model.score_with_y_pred_proba(y=y_val_fold, y_pred_proba=y_pred_proba)
(_dystack pid=6784)   File "/usr/local/lib/python3.10/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1063, in score_with_y_pred_proba
(_dystack pid=6784)     return compute_weighted_metric(y, y_pred, metric, sample_weight, quantile_levels=self.quantile_levels)
(_dystack pid=6784)   File "/usr/local/lib/python3.10/dist-packages/autogluon/core/utils/utils.py", line 673, i

*** Summary of fit() ***
Estimated performance of each model:
                           model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        RandomForestEntr_BAG_L2   0.885251     roc_auc       2.919253  11.495111                1.536745           5.069830            2       True          7
1            WeightedEnsemble_L3   0.885251     roc_auc       2.925807  11.715008                0.006553           0.219897            3       True         10
2        RandomForestGini_BAG_L2   0.882353     roc_auc       1.725896  11.577699                0.343387           5.152418            2       True          6
3          ExtraTreesEntr_BAG_L1   0.881773     roc_auc       0.341126   1.324413                0.341126           1.324413            1       True          4
4            WeightedEnsemble_L2   0.881773     roc_auc       0.342194   1.415673                0.001068           0.091260            2 

In [3]:
directory = '/content/dataset/' # Define the directory containing the CSV files
test_identity = pd.read_csv(directory + 'test_identity.csv')
test_transaction = pd.read_csv(directory + 'test_transaction.csv')

# Merge test datasets
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [4]:
y_predproba = predictor.predict_proba(test_data)


Loading: /content/AutoGluonModels/models/ExtraTreesEntr_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/ExtraTreesGini_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestEntr_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestGini_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestEntr_BAG_L2_FULL/model.pkl


In [7]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba.iloc[:, 1]
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)


In [8]:
submission.head()

,TransactionID,isFraud
0,3663549,0.003333
1,3663550,0.000000
2,3663551,0.000000
3,3663552,0.000000
4,3663553,0.000000


In [9]:
submission.to_csv('/content/dataset/my_submission.csv', index=False)


In [10]:
from google.colab import files
files.download('/content/dataset/my_submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>